# Annotate omics features for lncRNAs

## Step1 Get genomic position for lncRNAs

### Step1.1 Convert 0-based coordination to 1-based coordination.

In [ ]:
import pandas as pd

inter_lnc = pd.read_csv('../../data/LPI/mouse/lncRNA.csv')
inter_lnc = inter_lnc[['lncRNA_ID', 'chr', 'start', 'end', 'strand']]

inter_lnc.to_csv('lnc_with_position_0-based.csv', index=False) 

# convert 0-based position to 1-based position
inter_lnc['start'] = inter_lnc['start'] + 1

inter_lnc.to_csv('lnc_with_position_1-based.csv', index=False)



## Step2 Annotate histone marks, DHS and CTCF binding site for lncRNAs

### Step2.1 Calculate features of epigenomic marks.

In [ ]:
import pyBigWig
import os
import pandas as pd
import numpy as np

# File paths
lncRNA_file = "lnc_with_position_0-based.csv"
epi_folder = "../../omics/ENCODE_annotation/mouse/"


# Read lncRNA coordinate information
lncRNAs = pd.read_csv(lncRNA_file)

# Store lncRNA IDs that failed to extract features
failed_lncRNA_ids = set()

# List of tissue and epigenetic features
tissues = ['heart', 'lung', 'brain']
epi_features = ['DHS', 'H3K4me3', 'H3K9me3', 'H3K27ac', 'H3K36me3', 'H3K27me3']

# Function to calculate both peak counts and max signal value
def calculate_features(row, bb):
    chrom = str(row["chr"])
    start = int(row["start"])
    end = int(row["end"])
    
    try:
        entries = bb.entries(chrom, start, end)  # Extract peak intervals
        if entries is None:
            return 0, 0  # No peaks, return (0, 0)
        
        peak_count = len(entries)  # Count peaks
        signal_values = []
        
        # Extract signalValues (assuming details is space/tab-separated and signalValue is at index 6)
        for entry in entries:
            details = entry[2]  # Get the details string
            details_parts = details.split()  # Split by space or tab
            if len(details_parts) > 6:  # Ensure there are enough fields
                signal_value = float(details_parts[6])  # NarrowPeak format: signalValue is usually at index 6
                signal_values.append(signal_value)

        max_signal_value = max(signal_values) if signal_values else 0  # Get max signalValue
        return peak_count, max_signal_value
    
    except Exception as e:
        return 0, 0  # Return (0,0) in case of failure

# Iterate through all tissue folders
for tissue in tissues:
    # Initialize DataFrame to store all features
    combined_features = lncRNAs.copy()
    cell_folder = os.path.join(epi_folder, tissue)
    if not os.path.isdir(cell_folder):
        print(f"Warning: Cell line folder {cell_folder} does not exist, skipping.")
    else:
        # Iterate through all epigenetic feature bigBed files
        for epi_feature in epi_features:
            bigbed_file = os.path.join(cell_folder, f"{epi_feature}.bigbed")
            if not os.path.exists(bigbed_file):
                print(f"Warning: File {bigbed_file} does not exist, skipping.")
                continue
            
            # Open the bigBed file
            try:
                bb = pyBigWig.open(bigbed_file)
            except Exception as e:
                print(f"Failed to open file {bigbed_file}, error: {e}")
                continue
            
            print(f"Processing {tissue} - {epi_feature} ...")
            
            # Calculate both features per lncRNA region
            results = lncRNAs.apply(calculate_features, axis=1, bb=bb)

            # Store results in two separate columns
            combined_features[f"{tissue}_{epi_feature}_peak_counts"] = results.apply(lambda x: x[0])  # Peak counts
            combined_features[f"{tissue}_{epi_feature}_max_signalValue"] = results.apply(lambda x: x[1])  # Max signal value
            
            # Close the bigBed file
            bb.close()

    output_file = f"{tissue}_epi.csv"

    # Remove unnecessary columns and save results
    combined_features = combined_features.drop(columns=['chr', 'start', 'end', 'strand'])
    combined_features.to_csv(output_file, index=False)


Processing brain - DHS ...
Processing brain - H3K4me3 ...
Processing brain - H3K9me3 ...
Processing brain - H3K27ac ...
Processing brain - H3K36me3 ...
Processing brain - H3K27me3 ...


## Step3 Get conversation score for lncRNAs.

### Step3.1 Calculate feature of conservation score.

In [ ]:
import pandas as pd
import pyBigWig
import numpy as np
import os

def get_scores(df, bw_path, score_name):
    """
    Extracts conservation scores from a bigWig file.

    Args:
        df (pd.DataFrame): DataFrame containing lncRNA information.
        bw_path (str): Path to the bigWig file.
        score_name (str): Name of the conservation score.

    Returns:
        pd.DataFrame: DataFrame with mean and max scores.
        set: Set of lncRNA_IDs without scores.
    """
    scores_df = df.copy()
    scores_df[f'{score_name}_mean_score'] = np.nan
    scores_df[f'{score_name}_max_score'] = np.nan

    no_score_ids = set()

    try:
        with pyBigWig.open(bw_path) as bw:
            for index, row in df.iterrows():
                chrom = row['chr']
                if chrom in bw.chroms():
                    scores = bw.values(chrom, int(row['start']), int(row['end']), numpy=True)
                    if scores.size > 0 and not np.all(np.isnan(scores)):
                        scores_df.at[index, f'{score_name}_mean_score'] = np.nanmean(scores)
                        scores_df.at[index, f'{score_name}_max_score'] = np.nanmax(scores)
                    else:
                        no_score_ids.add(row['lncRNA_ID'])
                else:
                    no_score_ids.add(row['lncRNA_ID'])
    except RuntimeError as e:
        print(f"Failed to open {bw_path}: {e}")

    return scores_df, no_score_ids

def calculate_conservation_features(lncRNAs_csv, conservation_dir):
    """
    Calculates conservation features from all bigWig files in a given directory.

    Args:
        lncRNAs_csv (str): Path to lncRNA CSV file.
        conservation_dir (str): Directory containing bigWig files.

    Outputs:
        - lnc_with_conservation_scores.csv: lncRNAs with conservation scores.
        - lnc_without_conservation_scores.csv: lncRNAs without available scores.
    """
    lncRNAs = pd.read_csv(lncRNAs_csv)
    missing_scores_ids = set()

    # List all bigWig files in the directory
    bw_files = [f for f in os.listdir(conservation_dir) if f.endswith(('.bw', '.bigWig'))]

    if not bw_files:
        print(" No bigWig files found in the directory.")
        return

    # Initialize lnc_with_score with lncRNA IDs
    lnc_with_score = lncRNAs.copy()

    # Process each bigWig file
    for bw_file in bw_files:
        bw_path = os.path.join(conservation_dir, bw_file)
        score_name = os.path.splitext(bw_file)[0]  # Use filename as score name

        print(f"Processing {bw_file} ...")

        scores_df, no_score_ids = get_scores(lnc_with_score, bw_path, score_name)
        missing_scores_ids.update(no_score_ids)

        # Merge the new scores into the main DataFrame
        lnc_with_score = pd.merge(lnc_with_score, scores_df[[ 'lncRNA_ID', 
                                                              f'{score_name}_mean_score', 
                                                              f'{score_name}_max_score']], 
                                  on='lncRNA_ID', how='left')

    # Remove lncRNAs without any scores
    lnc_with_score = lnc_with_score.drop(columns=['chr', 'start', 'end', 'strand'])
    lnc_with_score = lnc_with_score[~lnc_with_score['lncRNA_ID'].isin(missing_scores_ids)]

    # Save outputs
    lnc_with_score.to_csv("conservation_feature.csv", index=False)
    pd.Series(list(missing_scores_ids)).to_csv("no_conservation_scores.csv", index=False, header=["lncRNA_ID"])

    print("Conservation scores calculated and saved.")
    print(f"conservation_feature.csv contains {len(lnc_with_score)} records.")
    print(f"no_conservation_scores.csv contains {len(missing_scores_ids)} missing lncRNA IDs.")

# Usage Example
conservation_dir = '../../omics/conservation/mouse'  # Directory containing multiple bigWig files
calculate_conservation_features('lnc_with_position_1-based.csv', conservation_dir)


Processing phyloP.bw ...
Processing phastCons.bw ...
Conservation scores calculated and saved.
conservation_scores.csv contains 37854 records.
no_conservation_scores.csv contains 279 missing lncRNA IDs.


## Step4 Get sequence feature

### Step4.1 Modify chromosome names to GENCODE standard names.

In [18]:
import pandas as pd

chrname_mapping = pd.read_csv("chromosome_name_mapping.csv")
lncRNA = pd.read_csv("lnc_with_position_1-based.csv")
chrname_dict = dict(zip(chrname_mapping['original_name'], chrname_mapping['standard_name']))
lncRNA['chr'] = lncRNA['chr'].replace(chrname_dict)

lncRNA.to_csv("lnc_with_standard_chrname.csv", index=False)

### Step4.2 Get sequence for lncRNAs.

In [ ]:
from pyfaidx import Fasta
import pandas as pd

# file path
genome_fasta = "../../reference_lncRNA/reference_genome/GRCm38.p6.genome.fa"
lncRNA_file = "lnc_with_standard_chrname.csv"
output_fasta = "lncRNA_sequences.fasta"

# load reference genome
genome = Fasta(genome_fasta)

# load lncRNAs 
lncRNA_coords = pd.read_csv(lncRNA_file)

# open output file
with open(output_fasta, "w") as fasta_out:
    for index, row in lncRNA_coords.iterrows():
        chrom = row["chr"]

        start = int(row["start"])
        end = int(row["end"])
        strand = row["strand"]
        lncRNA_id = row["lncRNA_ID"]

        # extract sequence
        try:
            seq = genome[chrom][start:end].seq
            if strand == "-":
                # extract complementary sequence if strand == "-"
                seq = genome[chrom][start:end].reverse.complement.seq

            fasta_out.write(f">{lncRNA_id}\n")
            fasta_out.write(seq + "\n")

        except KeyError:
            print(f"Chromosome {chrom} not found in the genome file.")


### Step4.2 Calculate sequence feature.

In [19]:
import re
import pandas as pd
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

def fasta_parser(fasta_file):
    """
    Custom FASTA file parser to read sequences manually.
    """
    with open(fasta_file, "r") as file:
        identifier = None
        sequence_lines = []
        for line in file:
            if line.startswith(">"):
                if identifier is not None:  # Save the previous sequence before reading a new one
                    yield SeqRecord(Seq(''.join(sequence_lines).replace("\n", "")), id=identifier.strip(), description='')
                identifier = line[1:].strip()  # Extract new sequence ID, removing '>'
                sequence_lines = []  # Reset sequence content
            else:
                sequence_lines.append(line.strip())
        if identifier is not None:  # Save the last sequence
            yield SeqRecord(Seq(''.join(sequence_lines).replace("\n", "")), id=identifier.strip(), description='')

# Specify the input FASTA file
input_fasta = "lncRNA_sequences.fasta"  # Replace with the actual FASTA file path

def calculate_cpg_and_gc(sequence, window_size=None):
    """
    Calculate CpG count, CpG islands, GC content, and sequence length for a given sequence.
    """
    sequence = sequence.upper()  # Ensure sequence is in uppercase
    seq_len = len(sequence)
    if window_size is None:
        cpg_count = sequence.count('CG')
        cpg_islands = len(re.findall(r'CG(CG)+', sequence))
        gc_content = (sequence.count('G') + sequence.count('C')) / seq_len if seq_len > 0 else 0
        return {
            "CpG_count": cpg_count,
            "CpG_islands": cpg_islands,
            "GC_content": round(gc_content * 100, 2),
            "Length": seq_len
        }

# Iterate through each sequence and compute features
results = []
for record in fasta_parser(input_fasta):
    seq_id = record.id
    sequence = str(record.seq)
    features = calculate_cpg_and_gc(sequence)  # Compute features for the entire sequence
    results.append({
        "lncRNA_ID": seq_id,
        "CpG_count": features["CpG_count"],
        "CpG_islands": features["CpG_islands"],
        "GC_content (%)": features["GC_content"],
        "Length": features["Length"]
    })

# Save computed features to a CSV file
df = pd.DataFrame(results)
output_csv = "seq_feature.csv"  # Output feature table
df.to_csv(output_csv, index=False)
print(f"Results saved to {output_csv}")


Results saved to seq_feature.csv


## Step5 Merge all features

In [ ]:
import pandas as pd

tissues = ['heart', 'lung', 'brain']

conservation_feature = pd.read_csv('conservation_feature.csv')
seq_feature = pd.read_csv('seq_feature.csv')
merged_feature = pd.merge(seq_feature, conservation_feature, on="lncRNA_ID", how="inner")
all_epi = merged_feature[['lncRNA_ID']]
for tissue in tissues:
	epi_feature = pd.read_csv(f'{tissue}_epi.csv')
	ot_merged_feature = pd.merge(merged_feature, epi_feature, on="lncRNA_ID", how="inner")
	all_epi = pd.merge(all_epi, epi_feature, on='lncRNA_ID', how='inner')
	ot_merged_feature.to_csv(f"{tissue}_annotation.csv", index=False)



## Step6 Filter out invalid interactions.

In [2]:
import pandas as pd

# File paths
inter_file = '../../data/LPPI/mouse/LPPI_updated.csv'
lncRNA_annotation_file = "heart_annotation.csv"
output_file = "inter_with_valid_lnc.csv"

# Load data
inter = pd.read_csv(inter_file)
lncRNA_with_annotation = pd.read_csv(lncRNA_annotation_file)

# Convert lncRNA_ID list to a set for faster lookup
lncRNA_set = set(lncRNA_with_annotation['lncRNA_ID'])

# Apply filtering
def filter_node_i(node):
    if str(node).startswith("l"):
        return node in lncRNA_set  # Check only if it starts with 'l'
    return True  # Keep all other nodes

valid_inter = inter[inter['Node_i'].apply(filter_node_i)]

# Save results
valid_inter.to_csv(output_file, index=False)
